<a href="https://colab.research.google.com/github/subhro-ds/langchain/blob/main/Langchain_chat_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Langchain OpenAI Chat Models Basic

1.   Set up langchain openai api
2.   Basic calling of openai api and invoke
3.   Basic conversation



In [1]:
%pip install langchain_openai --quiet
%pip install langchain_google_firestore --quiet
%pip install firebase-admin --quiet

In [4]:
from langchain_openai import ChatOpenAI
import os
from google.colab import userdata
from langchain_core.messages import AIMessage, SystemMessage, HumanMessage

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [5]:
llm = ChatOpenAI(temperature=0,max_retries=2,model="gpt-4o-mini")
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
print(ai_msg.content)
(ai_msg)

J'aime la programmation.


AIMessage(content="J'aime la programmation.", response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 31, 'total_tokens': 36}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_8b761cb050', 'finish_reason': 'stop', 'logprobs': None}, id='run-849a6995-7ef3-449b-aabb-7d5dfa24187a-0', usage_metadata={'input_tokens': 31, 'output_tokens': 5, 'total_tokens': 36})

In [5]:
# Messages are of 3 types - System Message, Human Message, AI Message
# system message always come first and then human message
# system message sets the context of the conversation
# human message is the human chatting with the AI
# AI message is the AI responding back to human message


messages = [
    SystemMessage(content="You are a helpful assistant that translates English to Spanish. Translate the user sentence."),
    HumanMessage(content="I love programming.")
]
result = llm.invoke(messages)
print(result.content)

Me encanta programar.


In [6]:
messages = [
    SystemMessage(content="You are a helpful assistant that translates English to Spanish. Translate the user sentence."),
    HumanMessage(content="I love programming.")
]

## Langchain chat model conversation


In [7]:
model = ChatOpenAI()

In [8]:
chat_history=[]

In [9]:
system_message = SystemMessage(content="You are a helpful AI Assistant.")
chat_history.append(system_message)

while True:
  query = input("You: ")
  if query.lower() == "exit":
    break
  chat_history.append(HumanMessage(content=query))
  result = model.invoke(chat_history)
  response = result.content
  chat_history.append(AIMessage(content=response))
  print(f"AI: {response}")

print("---- Message History ----")
print(chat_history)

You: What is Samosa?
AI: Samosa is a popular Indian snack that consists of a triangular or cone-shaped pastry filled with a savory mixture, typically including spiced potatoes, peas, and sometimes meat. It is usually deep-fried until crispy and is often served with chutney or sauce. Samosas are a delicious and popular street food in India and are enjoyed by people all over the world.
You: Can you please tell me a simple recipe of it?
AI: Sure! Here is a simple recipe for vegetarian samosas:

Ingredients:
- 2 large potatoes, boiled, peeled, and mashed
- 1/2 cup green peas, boiled
- 1 small onion, finely chopped
- 2-3 green chilies, finely chopped
- 1 tsp cumin seeds
- 1 tsp garam masala
- 1 tsp coriander powder
- 1/2 tsp turmeric powder
- Salt to taste
- Oil for frying
- Samosa wrappers or pastry sheets

Instructions:
1. Heat oil in a pan and add cumin seeds. Once they splutter, add chopped onions and green chilies. Saute until onions are translucent.
2. Add turmeric powder, garam masal

## Saving the messages to the cloud!

In [19]:
from google.cloud import firestore
from langchain_google_firestore import FirestoreChatMessageHistory
from langchain_openai import ChatOpenAI

In [26]:
import firebase_admin
from firebase_admin import credentials

# Check if a Firebase app is already initialized
if not firebase_admin._apps:
    # cred = credentials.Certificate("path/to/serviceAccountKey.json")
    cred = credentials.Certificate("/content/drive/MyDrive/Colab Notebooks/Langchain/serviceAccountKey.json")
    firebase_admin.initialize_app(cred)
else:
    print("Firebase app already initialized.")

Firebase app already initialized.


In [33]:
# Authenticate with Google Cloud (Important for Colab)
# This step is often required in Colab to grant access to Firestore
from google.colab import auth
auth.authenticate_user()

In [34]:
# Setup Firebase Firestore
PROJECT_ID = "langchain-chat-2164a"
SESSION_ID = "user_session_1"  # This could be a username or a unique ID
COLLECTION_NAME = "chat_history"
# Initialize Firestore Client
print("Initializing Firestore Client...")
client = firestore.Client(project=PROJECT_ID)

Initializing Firestore Client...


In [35]:
# Get a list of collections
collections = [collection.id for collection in client.collections()]
print(collections)

['chat_history']


In [38]:
# Initialize Firestore Chat Message History
print("Initializing Firestore Chat Message History...")
chat_history = FirestoreChatMessageHistory(
  session_id=SESSION_ID,
  collection=COLLECTION_NAME,
  client=client,
)
print("Chat History Initialized.")
print("Current Chat History:", chat_history.messages)


Initializing Firestore Chat Message History...
Chat History Initialized.
Current Chat History: [HumanMessage(content='what is samosa?'), AIMessage(content='Samosa is a popular Indian snack that consists of a deep-fried or baked pastry filled with a savory filling, typically featuring spiced potatoes, peas, and sometimes minced meat or lentils. It is usually triangular in shape and is often served with chutney or sauce for dipping. Samosas are commonly enjoyed as a street food and appetizer in South Asian cuisine.'), HumanMessage(content='write 3 lines of peotry on samosa'), AIMessage(content="Crispy, golden, filled with delight,\nSamosa's spicy bite, a savory sight.\nA taste of India, wrapped up tight.")]


In [39]:
# Initialize Chat Model
model = ChatOpenAI()

print("Start chatting with the AI. Type 'exit' to quit.")

while True:
  human_input = input("User: ")
  if human_input.lower() == "exit":
    break

  chat_history.add_user_message(human_input)

  ai_response = model.invoke(chat_history.messages)
  chat_history.add_ai_message(ai_response.content)

  print(f"AI: {ai_response.content}")

Start chatting with the AI. Type 'exit' to quit.
User: what was the previous chat about?
AI: The previous chat was about samosas, a popular Indian snack consisting of a deep-fried or baked pastry filled with a savory filling.
User: what are french fries?
AI: French fries, also known as fries, chips, or finger chips, are deep-fried potato strips that are typically served as a side dish or snack. They are commonly salted and may be served with ketchup, mayonnaise, or other condiments. French fries are a popular fast food item and are enjoyed in many countries around the world.
User: which country was it originated?
AI: French fries are believed to have originated in Belgium, not France as the name might suggest. They are said to have been created in the late 17th century in the region that is now known as Belgium, where villagers would fry small fish to eat, as the rivers would freeze over in the winter and they were unable to fish. When the rivers thawed, they started cutting potatoes i